In [1]:
import pandas as pd
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras import layers
from keras_tuner.tuners import RandomSearch
from keras._tf_keras.keras.optimizers import Adam

In [2]:
df = pd.read_csv("Real_Combine.csv")
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [3]:
df = df.dropna()

In [4]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]

In [5]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers',2, 20)):
        model.add(layers.Dense(units = hp.Int("units_" + str(i),
                                              min_value=32,
                                              max_value=512,
                                              step=32),
                                              activation="relu"))
    model.add(layers.Dense(units=1, activation="linear"))
    model.compile(
        optimizer=Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
            loss = "mean_absolute_error",
            metrics = ["mean_absolute_error"])
    return model

In [6]:
tuner = RandomSearch(
    build_model,
    objective="val_mean_absolute_error",
    max_trials=5,
    executions_per_trial = 3,
    directory = "project",
    project_name = "Air Quality Index"
    
)

Reloading Tuner from project\Air Quality Index\tuner0.json


In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 17
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [8]:
from sklearn.model_selection import train_test_split


x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [9]:
tuner.search(x_train,y_train,epochs=5,
             validation_data=(x_test,y_test))


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
10                |10                |num_layers
96                |96                |units_0
512               |512               |units_1
0.001             |0.001             |learning_rate
320               |320               |units_2
384               |384               |units_3
416               |416               |units_4
448               |448               |units_5
448               |448               |units_6
256               |256               |units_7
416               |416               |units_8
224               |224               |units_9
320               |320               |units_10
128               |128               |units_11
160               |160               |units_12
448               |448               |units_13
32                |32                |units_14


Epoch 1/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - loss: 77.9264 - mean_absolute_error: 77.9264 - val_loss: 64.1352 - va

e:\Learning DL\dl\Lib\site-packages\keras_tuner\src\engine\metrics_tracking.py:111: RuntimeWarning: All-NaN axis encountered
  np.nanmin(values) if self.direction == "min" else np.nanmax(values)


RuntimeError: Number of consecutive failures exceeded the limit of 3.
